# Data Description

- Training set: 8000 images in total
    - Dogs: 4000 images
    - Cats: 4000 images

- Test set: 2000 images in total
    - Dogs: 1000 images
    - Cats: 1000 images

# 1. Importing the Libraries

- `tensorflow`

- preprocessing module of `keras` (for image pre-processing)
    - `tensorflow.keras.preprocessing.image` module has a `ImageDataGenerator` class &rarr; [docs](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.10.0'

# 2. Data Preprocessing

- We preprocess the dataset of images.

## 2.1. Preprocessing the Training set

- We will apply some transformations on all images of only the training set.
    - To avoid overfitting.
    - We apply some simple geometric transformations like transvections, rotate images, do some zooms, and so on.
    - This will 'augment' the images.
    - This is called 'Image Augmentation'.
    - `ImageDataGenerator` class of `tensorflow.keras.preprocessing.image` module does this work.

- `rescale` parameter does Feature Scaling for each pixel value (divides by 255 in our example).
    - Feature Scaling is compulsory for NN.
    
- Refer to `tf.keras` documentation for details of this class and code snippets.

In [3]:
# create the Training Image Data Generator
train_datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    shear_range = 0.2, 
    zoom_range = 0.2,
    horizontal_flip = True
)

# connect it to the training set
training_set = train_datagen.flow_from_directory(
    'dataset/training_set', # path of training set
    target_size = (64, 64), # final size of the images, which will be fed into CNN
    batch_size = 32, # number of images in each batch
    class_mode = 'binary' # can be binary or categorical
)

Found 8000 images belonging to 2 classes.


## 2.2. Preprocessing the Test set

- For test images, we dont apply image augmentations. But, they should be rescaled (pixels).

- for `predict` method, images must have same dimensions. So, make use of same `target_size` in training and test datasets.

In [4]:
# create the Test set Image Data Generator
test_datagen = ImageDataGenerator(rescale = 1./255)

# connect it to the test set
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size = (64, 64), # should be of same size as in training set
        batch_size = 32, 
        class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.


# 3. Building the CNN

- Lets build the architecture of the CNN.

## 3.1. Initialising the CNN

- A CNN is still a sequence of layers, so make use of `Sequential` class.

- Then, step by step, add layers to the `cnn` model object.

In [5]:
from tensorflow.keras.models import Sequential

cnn = Sequential()

### 3.1. Step 1 - Convolution

- Convolution Layer is an instance of `tensorflow.keras.layers` module's `Conv2D` class.

- Constructor arguments:
    - Filter or Feature Detector or Kernel: `filters`
        - Classic Architecture: 32 filters in first conv layers, and 32 filters in 2nd conv layer
    - Kernel Size: `kernel_size` (Size of feature detector. Say 3 &rarr; its a 3*3 matrix)
    - activation function: `activation`
    - `input_shape` &rarr; while adding any layers, you should specify the input size.
        - Color images &rarr; RGB &rarr; 3D input shape.
        - Since images are resized to 64*64 px &rarr; `[64, 64, 3]` is the input shape.
        - This is only for the input layer (the 1st conv layer)

In [6]:
from tensorflow.keras.layers import Conv2D

cnn.add(
    Conv2D(
        filters = 32,
        kernel_size = 3, # 3*3
        activation = 'relu',
        input_shape = [64, 64, 3]
    )
)

### 3.1. Step 2 - Pooling

- Lets apply max pooling.

- Its actually a pooling layer.

- `tensorflow.keras.layers` has a `MaxPool2D` class.

- 2 args:
    - `pool_size` (that square frame width)
        - Recommended pool size in max pooling is 2
    - `strides` (jump size of the frame)
        - Recommended stride is 2

In [7]:
from tensorflow.keras.layers import MaxPool2D

cnn.add(
    MaxPool2D(
        pool_size = 2,
        strides = 2
    )
)

## 3.2. Adding a second convolutional layer

In [8]:
# add a 2nd conv layer
cnn.add(
    Conv2D(
        filters = 32,
        kernel_size = 3, # 3*3
        activation = 'relu',
    )
)

cnn.add(
    MaxPool2D(
        pool_size = 2,
        strides = 2
    )
)

### 3.2. Step 3 - Flattening

- Flatten the result into a 1D vector which will be the input to an ANN.

- Flattening layer
    - `tensorflow.keras.layers` module has a `Flatten` class

In [9]:
from tensorflow.keras.layers import Flatten

cnn.add(Flatten())

### 3.2. Step 4 - Full Connection

- Add a fully connected (hidden) layer: `Dense` layer

- For Computer Vision applications, use some bigger number of neurons.

In [10]:
from tensorflow.keras.layers import Dense

cnn.add(
    Dense(
        units = 128,
        activation = 'relu'
    )
)

### 3.2. Step 5 - Output Layer

- Fully connected to the hidden layer

- Number of units in output layer = 1 for binary classification and use sigmoid activation

- Number of units in output layer = number of output classes and use softmax activation for multiclass classification

In [11]:
cnn.add(
    Dense(
        units = 1,
        activation = 'sigmoid'
    )
)

# 4. Training the CNN

In [12]:
# get a summary of the model so far
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

- we do a different kind of training.

- lets train the NN for 25 epochs.
    - for each epoch, we will see the test set results
    
- each epoch takes some considerable time to execute. so try different values until you see some sort of convergence
    
- Adam optimizer to perform stochastic gradient descent to update weights

- BinaryCrossEntropy loss function

- accuracy metrics to measure the performance

- training takes lot of time, depending on your machine

In [13]:
# compile the model
cnn.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [14]:
# train the cnn, evaluate test set at each epoch
cnn.fit(
    x = training_set,
    validation_data = test_set, # on which we want to evaluate and validate
    epochs = 25
)

Epoch 1/25
250/250 [==============================] - 112s 434ms/step - loss: 0.6755 - accuracy: 0.5724 - val_loss: 0.6225 - val_accuracy: 0.6505
Epoch 2/25
250/250 [==============================] - 34s 137ms/step - loss: 0.6204 - accuracy: 0.6536 - val_loss: 0.6538 - val_accuracy: 0.6345
Epoch 3/25
250/250 [==============================] - 35s 142ms/step - loss: 0.5896 - accuracy: 0.6930 - val_loss: 0.5729 - val_accuracy: 0.7125
Epoch 4/25
250/250 [==============================] - 37s 150ms/step - loss: 0.5557 - accuracy: 0.7125 - val_loss: 0.5183 - val_accuracy: 0.7445
Epoch 5/25
250/250 [==============================] - 39s 156ms/step - loss: 0.5200 - accuracy: 0.7476 - val_loss: 0.5328 - val_accuracy: 0.7465
Epoch 6/25
250/250 [==============================] - 36s 144ms/step - loss: 0.4984 - accuracy: 0.7594 - val_loss: 0.4738 - val_accuracy: 0.7805
Epoch 7/25
250/250 [==============================] - 37s 148ms/step - loss: 0.4735 - accuracy: 0.7747 - val_loss: 0.4960 - val_a

# 5. Making a single prediction

- import `numpy` library.

- import `tensorflow.keras.preprocessing.image` module

- load the image, which acts as input to `predict` method
    - input image should have same size as the images used in training
    - `image.load_image(path_to_image_from_root_with_extension, target_size = input_img_size)`
    
- this returns a `PIL` format. `predict` method requires an array. so, do that conversion

- then, since we created batches of 32 images during cnn training. so, all images during predictions should be in batches
    - dimension of the batch is always the first dimension

In [16]:
import numpy as np
from tensorflow.keras.preprocessing import image

# load the image (PIL)
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))

# convert into array (PIL -> numpy array)
test_image = image.img_to_array(test_image)

# add an extra dimension (1st dimension -> axis = 0), converting the image into a batch
test_image = np.expand_dims(test_image, axis = 0)

In [17]:
# predict the results (0 or 1)
result = cnn.predict(test_image)
print(result)

# encode the results (what does 0 repr? dog or cat?)
print(training_set.class_indices)

# display the result in a good format
if result[0][0] == 1:
    prediction = 'Dog'
else:
    prediction = 'Cat'
    
print(prediction)

1/1 [==============================] - 3s 3s/step
[[1.]]
{'cats': 0, 'dogs': 1}
Dog


In [18]:
# TESTING FOR CAT

# load the image (PIL)
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))

# convert into array (PIL -> numpy array)
test_image = image.img_to_array(test_image)

# add an extra dimension (1st dimension -> axis = 0), converting the image into a batch
test_image = np.expand_dims(test_image, axis = 0)

# predict the results (0 or 1)
result = cnn.predict(test_image)
print(result)

# encode the results (what does 0 repr? dog or cat?)
print(training_set.class_indices)

# display the result in a good format
if result[0][0] == 1:
    prediction = 'Dog'
else:
    prediction = 'Cat'
    
print(prediction)

1/1 [==============================] - 0s 20ms/step
[[5.929237e-38]]
{'cats': 0, 'dogs': 1}
Cat
